In [1]:
#load NLP Packages
import spacy
nlp = spacy.load('en_core_web_sm')

In [2]:
#Explore Components of NLP pipeline
#Method
nlp.components

[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec at 0x1305e5dc0>),
 ('tagger', <spacy.pipeline.tagger.Tagger at 0x1305e5ca0>),
 ('parser', <spacy.pipeline.dep_parser.DependencyParser at 0x1304f4eb0>),
 ('senter', <spacy.pipeline.senter.SentenceRecognizer at 0x1305e5a60>),
 ('attribute_ruler',
  <spacy.pipeline.attributeruler.AttributeRuler at 0x13076c300>),
 ('lemmatizer', <spacy.lang.en.lemmatizer.EnglishLemmatizer at 0x1308077c0>),
 ('ner', <spacy.pipeline.ner.EntityRecognizer at 0x1304f4e40>)]

In [3]:
#Method 2
nlp.pipeline

[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec at 0x1305e5dc0>),
 ('tagger', <spacy.pipeline.tagger.Tagger at 0x1305e5ca0>),
 ('parser', <spacy.pipeline.dep_parser.DependencyParser at 0x1304f4eb0>),
 ('attribute_ruler',
  <spacy.pipeline.attributeruler.AttributeRuler at 0x13076c300>),
 ('lemmatizer', <spacy.lang.en.lemmatizer.EnglishLemmatizer at 0x1308077c0>),
 ('ner', <spacy.pipeline.ner.EntityRecognizer at 0x1304f4e40>)]

Using Spacy Textblob: Sentiment Analysis using Textblob

In [4]:
from spacytextblob.spacytextblob import SpacyTextBlob

In [5]:
# Adding SpacyTextblob to NLP Pipeline
nlp.add_pipe("spacytextblob")

In [6]:
# Recheck our components
nlp.components

[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec at 0x1305e5dc0>),
 ('tagger', <spacy.pipeline.tagger.Tagger at 0x1305e5ca0>),
 ('parser', <spacy.pipeline.dep_parser.DependencyParser at 0x1304f4eb0>),
 ('senter', <spacy.pipeline.senter.SentenceRecognizer at 0x1305e5a60>),
 ('attribute_ruler',
  <spacy.pipeline.attributeruler.AttributeRuler at 0x13076c300>),
 ('lemmatizer', <spacy.lang.en.lemmatizer.EnglishLemmatizer at 0x1308077c0>),
 ('ner', <spacy.pipeline.ner.EntityRecognizer at 0x1304f4e40>),
 ('spacytextblob', <spacytextblob.spacytextblob.SpacyTextBlob at 0x13222e2b0>)]

In [7]:
# Recheck 
nlp.pipeline

[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec at 0x1305e5dc0>),
 ('tagger', <spacy.pipeline.tagger.Tagger at 0x1305e5ca0>),
 ('parser', <spacy.pipeline.dep_parser.DependencyParser at 0x1304f4eb0>),
 ('attribute_ruler',
  <spacy.pipeline.attributeruler.AttributeRuler at 0x13076c300>),
 ('lemmatizer', <spacy.lang.en.lemmatizer.EnglishLemmatizer at 0x1308077c0>),
 ('ner', <spacy.pipeline.ner.EntityRecognizer at 0x1304f4e40>),
 ('spacytextblob', <spacytextblob.spacytextblob.SpacyTextBlob at 0x13222e2b0>)]

In [8]:
mytext = "I love chocolate"
docx = nlp(mytext)

In [9]:
#check sentiment polarity
docx._.polarity

0.5

In [10]:
#check sentiment subjectivity
docx._.subjectivity

0.6

In [11]:
#check assessment: list polarity/subjectivity for the assessed token
docx._.assessments

[(['love'], 0.5, 0.6, None)]

Applying Automation for Sentiment Analysis to all reviews:

We will be doing so with one list at a time for less running time and workload.

In [ ]:
import pandas as pd
df = pd.read_csv("data/review/reviews_list1.csv")

#automate polarity for reviews2.csv
def findSentiment():
    for i, row in df.iterrows():
        comments = str(df.at[i, 'Reviews'])
        polarity = nlp(comments)._.polarity
        '''
        #test break
        if i==8:
            break
        
        '''
        #if statement for pos/neg/na
        if polarity > 0:
            rating = "positive"
        elif polarity < 0:
            rating = "negative"
        elif polarity == 0:
            rating = "NA"
        #add new column
        df.at[i, 'Sentiment Rating'] = rating
findSentiment()
#export to new csv file
df.to_csv('output CSV files/ratedReviews_1.csv')

In [ ]:
df = pd.read_csv("data/review/reviews_list1_5.csv")
findSentiment()
df.to_csv('output CSV files/ratedReviews_1_5.csv')

In [21]:
df = pd.read_csv("data/review/reviews_list2.csv")
findSentiment()
df.to_csv('output CSV files/ratedReviews_2.csv')

In [22]:
df = pd.read_csv("data/review/reviews_list3.csv")
findSentiment()
df.to_csv('output CSV files/ratedReviews_3.csv')

In [23]:
df = pd.read_csv("data/review/reviews_list4.csv")
findSentiment()
df.to_csv('output CSV files/ratedReviews_4.csv')

In [24]:
df = pd.read_csv("data/review/reviews_list5.csv")
findSentiment()
df.to_csv('output CSV files/ratedReviews_5.csv')

In [25]:
df = pd.read_csv("data/review/reviews_list6.csv")
findSentiment()
df.to_csv('output CSV files/ratedReviews_6.csv')

In [26]:
df = pd.read_csv("data/review/reviews_list7.csv")
findSentiment()
df.to_csv('output CSV files/ratedReviews_7.csv')

In [27]:
df = pd.read_csv("data/review/reviews_list8.csv")
findSentiment()
df.to_csv('output CSV files/ratedReviews_8.csv')

In [28]:
df = pd.read_csv("data/review/reviews_list9.csv")
findSentiment()
df.to_csv('output CSV files/ratedReviews_9.csv')

In [ ]:
df = pd.read_csv("data/review/reviews_list10.csv")
findSentiment()
df.to_csv('output CSV files/ratedReviews_10.csv')

Open the output dataframes with the sentimental analysis

In [14]:
import pandas as pd

path = "/Users/sanzi/Desktop/project/AirBnB-Opinion-Mining-Project/output CSV files/"

df1 = pd.read_csv(path + "ratedReviews_1.csv")
df2 = pd.read_csv(path + "ratedReviews_1_5.csv")
df3 = pd.read_csv(path + "ratedReviews_2.csv")
df4 = pd.read_csv(path + "ratedReviews_3.csv")
df5 = pd.read_csv(path + "ratedReviews_4.csv")
df6 = pd.read_csv(path + "ratedReviews_5.csv")
df7 = pd.read_csv(path + "ratedReviews_6.csv")
df8 = pd.read_csv(path + "ratedReviews_7.csv")
df9 = pd.read_csv(path + "ratedReviews_8.csv")
df10 = pd.read_csv(path + "ratedReviews_9.csv")
df11 = pd.read_csv(path + "ratedReviews_10.csv")

Group counts and get the counts for each listing id

In [15]:
def countAnalyis(df):
        df_grouped = df.groupby(['listing_id','Sentiment Rating']).count()
        id = df_grouped.index.unique().tolist()
        rating = {}
        count_review = 0

        for i, row in df_grouped.iterrows():
                
                count_review = df_grouped.at[i, 'Reviews']
                
                listing_id_dict = i[0]
                
                review_type = i[1]
        
                #print("ID")
                #print(listing_id_dict)
                
                #print("Review")
                #print(review_type)
                
                #print("Count")
                #print(count_review)
                
                #print("-------")

                
                #  the listing id is not in the dictionary
        
                if listing_id_dict not in rating:
                        
                        #print("The listing ID is not in the dictionary")
                        
                        # Add the listing ID to the rating Dictionary with a list [0, 0, 0] : [P, Netural, Negative]
                        
                        rating[listing_id_dict] = [0, 0, 0]
                        
                        # The review type for this listing was Netural 
                        if review_type == "NA":
                                #print("Review type for this is NA")
                                #print(count_review)
                                rating.get(listing_id_dict)[1] = count_review

                                        
                        # The review type for this listing was Postive 
                        if review_type == "positive":
                                #print("Review type for this is Postive")
                                #print(count_review)
                                rating.get(listing_id_dict)[0] = count_review
                        
                                        
                        # The review type for this listing was Negative 
                        if review_type == "negative":
                                #print("Review type for this is Negative")
                                #print(count_review)
                                rating.get(listing_id_dict)[2] = count_review
                
                # else the listing id is in the dictionary
                else:
                        # The review type for this listing was Netural 
                        if review_type == "NA":
                                #print("Review type for this is NA")
                                #print(count_review)
                                rating.get(listing_id_dict)[1] = count_review

                                        
                        # The review type for this listing was Postive 
                        if review_type == "positive":
                                #print("Review type for this is Postive")
                                #print(count_review)
                                rating.get(listing_id_dict)[0] = count_review
                        
                                        
                        # The review type for this listing was Negative 
                        if review_type == "negative":
                                #print("Review type for this is Negative")
                                #print(count_review)
                                rating.get(listing_id_dict)[2] = count_review
                        
        return rating

Make the rating dictonary into a dataframe with listing_id as rows and columns as description

In [17]:
rating = countAnalyis(df1)
rating_df1 = pd.DataFrame({'Listing ID': list(rating.keys()), 'Description': list(rating.values())})


In [18]:
rating = countAnalyis(df2)
rating_df2 = pd.DataFrame({'Listing ID': list(rating.keys()), 'Description': list(rating.values())})

In [21]:
rating = countAnalyis(df3)
rating_df3 = pd.DataFrame({'Listing ID': list(rating.keys()), 'Description': list(rating.values())})

In [22]:
rating = countAnalyis(df4)
rating_df4 = pd.DataFrame({'Listing ID': list(rating.keys()), 'Description': list(rating.values())})

In [23]:
rating = countAnalyis(df5)
rating_df5 = pd.DataFrame({'Listing ID': list(rating.keys()), 'Description': list(rating.values())})

In [24]:
rating = countAnalyis(df6)
rating_df6 = pd.DataFrame({'Listing ID': list(rating.keys()), 'Description': list(rating.values())})

In [25]:
rating = countAnalyis(df7)
rating_df7 = pd.DataFrame({'Listing ID': list(rating.keys()), 'Description': list(rating.values())})

In [26]:
rating = countAnalyis(df8)
rating_df8 = pd.DataFrame({'Listing ID': list(rating.keys()), 'Description': list(rating.values())})

In [27]:
rating = countAnalyis(df9)
rating_df9 = pd.DataFrame({'Listing ID': list(rating.keys()), 'Description': list(rating.values())})

In [28]:
rating = countAnalyis(df10)
rating_df10 = pd.DataFrame({'Listing ID': list(rating.keys()), 'Description': list(rating.values())})

In [29]:
rating = countAnalyis(df11)
rating_df11 = pd.DataFrame({'Listing ID': list(rating.keys()), 'Description': list(rating.values())})

In [30]:
# Combine all the dataframes into one big dataframe
frames = [rating_df1, rating_df2, rating_df3, 
          rating_df4, rating_df5, rating_df6,
          rating_df7, rating_df8, rating_df9,
          rating_df10, rating_df11]
  
result = pd.concat(frames)

In [31]:
result

,Listing ID,Description
0,2539,"[8, 0, 0]"
1,2595,"[36, 0, 0]"
2,5121,"[47, 0, 0]"
3,5136,"[2, 0, 0]"
4,5178,"[390, 0, 11]"
...,...,...
7912,706487852182826405,"[1, 0, 0]"
7913,706611147133623424,"[1, 0, 0]"
7914,706712410572332671,"[1, 0, 0]"
7915,706752921121143414,"[1, 0, 0]"


Convert the Dataframe into the CSV

In [32]:
result.to_csv("/Users/sanzi/Desktop/project/AirBnB-Opinion-Mining-Project/output CSV files/Sentimental_Count.csv")

Merge listings.csv and Sentimental_Count.csv dataframes by an Inner Join.

It will merge both dataframes based on the specified column and then return new dataframe containing only those rows that have a matching value in both original dataframes.

In [5]:
import pandas as pd
# reading two csv files
data1 = pd.read_csv('data/listings.csv')
data2 = pd.read_csv('output CSV files/Sentimental_Count.csv')

# renaming column on data1 to match data2
data1.rename(columns={'id': 'Listing ID'}, inplace=True)
data1.set_index('Listing ID').to_csv("output CSV files/listingsRenameID.csv")

# using merge function by setting how='inner'
data3 = pd.read_csv('output CSV files/listingsRenameID.csv')
output = pd.merge(data3, data2, 
                   on='Listing ID', 
                   how='inner')
  
# output result
output.to_csv('C:/Users/Hello/Documents/GitHub/AirBnB-Opinion-Mining-Project/output CSV files/merged.csv')

Convert the CSV Dataframe into GeoJSON

In [7]:
import geopandas as gpd
from geopandas import GeoSeries, GeoDataFrame
from shapely import wkt

#Read new geocoded CSV file
d = pd.read_csv('output CSV files/merged.csv')

#CSV to geoJSON
df = pd.DataFrame(d)
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs="EPSG:4326")
gdf.to_file(filename='output geoJSON files/listingsRated.geojson', driver="GeoJSON")